In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# general
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import shutil
import pickle

# plotly
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import custom libraries
import sys
# sys.path.append("C:\\DATA\\Tasks\\lib\\hk")
# import hk_utils

# folder paths
ADD_DATA = "C:\\DATA\\data\\raw\\mimic4\\lookup\\"
ADD_DATA_proc = "C:/DATA/data/processed/"


PATH_PAPER = "C:\\DATA\\Tasks\\220704\\Alternate-Transactions-Articles-LaTeX-template\\images\\"


PATH_SYS="/scratch/hokarami/data_old/"

In [2]:
# libraries for THP

import argparse
import numpy as np
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter

import transformer.Constants as Constants
import Utils

# from preprocess.Dataset import get_dataloader, get_dataloader2
# from transformer.Models import Transformer
# from transformer.hk_transformer import Transformer
from tqdm import tqdm

# from torchinfo import summary

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# torch.cuda.empty_cache()
# torch.cuda.memory_allocated()
# torch.cuda.memory_reserved()

from sklearn import metrics
# from hk_pytorch import save_checkpoint,load_checkpoint
# import hk_pytorch


# from custom2 import myparser
import re

/opt/conda/envs/paper2022/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/paper2022/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import Main
import webbrowser


In [4]:
import io

In [5]:
from tsnecuda import TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE

In [6]:
# !pip install wandb -qqq
import wandb
# wandb.login()
api = wandb.Api()
import os

os.environ["WANDB_API_KEY"] = "0f780ac8a470afe6cb7fc474ff3794772c660465"

os.environ["WANDB_NOTEBOOK_NAME"] = "jup_res"

# Model path

In [80]:
runs_path=[
# "hokarami/TEEDAM_supervised/5uzxzoxd",
"hokarami/TEEDAM_supervised/sb9zf1mm",
# "hokarami/TEEDAM_unsupervised_timeCat/66pgiau2",

# "hokarami/TEEDAM_unsupervised_timeCat/j2ck0i9i",

]

run_path = runs_path[-1]
run_path


run = api.run(run_path)
for file in run.files():
    file.download(replace=True,root='./local/')

'hokarami/TEEDAM_supervised/sb9zf1mm'

<_io.TextIOWrapper name='./local/best_model.pkl' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='./local/conda-environment.yaml' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='./local/config.yaml' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='./local/opt.pkl' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='./local/output.log' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='./local/requirements.txt' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='./local/wandb-metadata.json' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='./local/wandb-summary.json' mode='r' encoding='UTF-8'>

In [81]:
# os.listdir(add_data)[8:]





# run_path = runs_path[-1]

# os.listdir(run_path)

# # load opt file


opt = pickle.load(open('./local/opt.pkl','rb'))
# modify opt
opt = Main.config(opt, justLoad=True)

checkpoint = torch.load('./local/best_model.pkl')
# # modify opt
# # opt = Main.config(opt, justLoad=True)




############################## CUDA True
[Info] INPUT TEDAM --> single-mark-label
[Info] parameters: Namespace(data='/home/hokarami/data/p12-seft/', data_label='multilabel', cuda=1, wandb=True, wandb_project='TEEDAM_supervised', log_freq=1, prof=False, per=100, balanced_batch=True, transfer_learning='', freeze='', ES_pat=100, setting='seft', test_center='', split='', log='log.txt', user_prefix='[R200-TEDA__pp_single_mark-concat]', pos_alpha=1.0, epoch=15, batch_size=128, lr=0.01, smooth=0.0, weight_decay=0.1, event_enc=1, time_enc='concat', te_d_mark=8, te_d_time=8, te_d_rnn=256, te_d_inner=16, te_d_k=8, te_d_v=8, te_n_head=4, te_n_layers=4, te_dropout=0.1, state=True, demo=True, num_states=1, noise=False, mod='single', int_dec='sahp', w_event=1.0, next_mark=1, w_class=False, w_pos=False, mark_detach=0, w_time=1.0, sample_label=1, w_pos_label=0.5, w_sample_label=100.0, hparams2write={'data': '/home/hokarami/data/p12/', 'data_label': 'multilabel', 'cuda': 1, 'wandb': True, 'wandb_projec

## load pre-trained model

In [82]:
model = Main.ATHP(
       n_marks=opt.num_marks,
        TE_config = opt.TE_config,
        DAM_config = opt.DAM_config,
        NOISE_config = opt.NOISE_config,

        CIF_config = opt.CIF_config,
        next_time_config = opt.next_time_config,
        next_type_config = opt.next_type_config,
        label_config = opt.label_config,

        demo_config = opt.demo_config,

        device=opt.device,



    )
model.to(opt.device)



# optimizer = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()),
#                            opt.lr, betas=(0.9, 0.999), eps=1e-05, weight_decay=opt.weight_decay)
    

model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# scheduler = optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.5)

_ = model.eval()

[info] STATE will be considered


ATHP(
  (TE): Encoder(
    (event_emb): Linear(in_features=25, out_features=8, bias=True)
    (layer_stack): ModuleList(
      (0): EncoderLayer(
        (slf_attn): MultiHeadAttention(
          (w_qs): Linear(in_features=16, out_features=32, bias=False)
          (w_ks): Linear(in_features=16, out_features=32, bias=False)
          (w_vs): Linear(in_features=16, out_features=32, bias=False)
          (fc): Linear(in_features=32, out_features=16, bias=True)
          (attention): ScaledDotProductAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (layer_norm): LayerNorm((16,), eps=1e-06, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (pos_ffn): PositionwiseFeedForward(
          (w_1): Linear(in_features=16, out_features=16, bias=True)
          (w_2): Linear(in_features=16, out_features=16, bias=True)
          (layer_norm): LayerNorm((16,), eps=1e-06, elementwise_affine=True)
          (dropout): Dropou

<All keys matched successfully>

In [83]:
dict_metrics,r_enc_list, out = Main.valid_epoch_tsne(model, opt.trainloader, opt.pred_loss_func, opt)

In [84]:
len(r_enc_list)


r_enc_list[0].shape
model.d_out_te
model.d_out_dam
X = np.concatenate(r_enc_list,axis=0)[:,:model.d_out_te+model.d_out_dam]
X = np.concatenate(r_enc_list,axis=0)[:,:]

X.shape

59

torch.Size([128, 52])

16

32

(7552, 52)

In [85]:
# y_state_pred = out['y_state_pred']
# y_state_true = out['y_state_true']
# y_state_score = out['y_state_score']

y_state_pred = None
y_state_true = None
y_state_score = None

y_pred = out['y_pred']
y_true = out['y_true']
y_score = out['y_score']

non_pad_mask_list = out['non_pad_mask_list']
event_type_list = out['event_type_list']


In [86]:
event_type_list[0].shape
non_pad_mask_list[0].shape

num_marks = event_type_list[0].shape[-1]


res = list()

for event_type, non_pad_mask in zip(event_type_list,non_pad_mask_list):

    temp = torch.unbind(event_type,0)
    lens = non_pad_mask.sum(1).long()

    for i,x in enumerate(temp):
        res.append( x[:lens[i],:].cpu().numpy().astype(np.uint8).transpose()*255 )

    

    
    # idd = non_pad_mask.sum(1).long()-1
    # idd = idd[:,None,None].repeat(1,1,num_marks)

    # torch.gather(event_type,1,idd).squeeze().shape

len(res)

torch.Size([128, 103, 25])

torch.Size([128, 103])

7552

In [87]:
from PIL import Image

all_images=[]

for i,pattern in enumerate(res):
    image = Image.fromarray(pattern)
    all_images.append(f'./local/images/img{i}.jpeg')
    image.save(f'./local/images/img{i}.jpeg')


# Visualizations

## TSNE of Learned Rep

In [88]:
TSNE_LIMIT = 6000



X.shape
all_colors = ["#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9",
"#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9",
"#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9",
"#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9",
"#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9",
"#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9",
"#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9",
"#52ef99", "#1c875c", "#69c8c1", "#4c707b", "#b6c5f5", "#1642cd", "#fb5de7", "#be64a7", "#62385e", "#edb1ff", "#860967", "#3d84e3", "#c5df72", "#6e3901", "#fba55c", "#9f2114", "#36e515", "#65a10e", "#474a09", "#f5603a", "#fa217f", "#4007d9"]

tsne = TSNE(n_components=2, perplexity=5, learning_rate=10,n_jobs=4)
X_tsne = tsne.fit_transform(X[:TSNE_LIMIT,:])
# colors_tsne = [all_colors[label] for label in dict_metrics['tsne']['y_true'][:TSNE_LIMIT] ]
X_tsne.shape


(7552, 52)

/opt/conda/envs/paper2022/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



(6000, 2)

In [90]:
df = pd.DataFrame()
df['x']=X_tsne[:,0]
df['y']=X_tsne[:,1]
df['color']=0
df['id']=np.arange(len(df))


if y_state_true:

    TP = (y_state_true[:TSNE_LIMIT]*y_state_pred[:TSNE_LIMIT])==1
    FP_FN = (y_state_true[:TSNE_LIMIT]+y_state_pred[:TSNE_LIMIT])==1
    FN = (y_state_true[:TSNE_LIMIT]-y_state_pred[:TSNE_LIMIT])==1
    FP = (y_state_true[:TSNE_LIMIT]-y_state_pred[:TSNE_LIMIT])==-1

    TN = (y_state_true[:TSNE_LIMIT]+y_state_pred[:TSNE_LIMIT])==0



    # df.loc[TN, 'color']='True Negatives'
    df.loc[TP, 'color']='True Positives'
    df.loc[FN, 'color']='False Negatives'
    df.loc[FP, 'color']='False Positives'
    df.loc[TN, 'color']='True Negatives'

    df.loc[y_state_true[:TSNE_LIMIT].astype(bool).flatten(), 'color_true']='Positive Samples'
    df.loc[~y_state_true[:TSNE_LIMIT].astype(bool).flatten(), 'color_true']='Negative Samples'


    df.loc[y_state_pred[:TSNE_LIMIT].astype(bool).flatten(), 'color_pred']='Positive Predicted'
    df.loc[~y_state_pred[:TSNE_LIMIT].astype(bool).flatten(), 'color_pred']='Negative Predicted'

# # df.loc[y_state_pred[:TSNE_LIMIT].astype(bool), 'color']='Negative Samples'

# # df['color_true']=y_state_true[:TSNE_LIMIT]
# # df['color_pred']=y_state_pred[:TSNE_LIMIT]

df['images']=all_images[:TSNE_LIMIT]



# fig = px.scatter(df,x='x',y='y',color='color', title='temp', hover_data=['id']).update_layout(width=800, height=600)
# fig.show()


# fig = px.scatter(df,x='x',y='y',color='color_pred', title='temp', hover_data=['id']).update_layout(width=800, height=600)
# fig.show()

# fig = px.scatter(df,x='x',y='y',color='color_true', title='temp', hover_data=['id']).update_layout(width=800, height=600)
# fig.show()
# fig.write_image(PATH_PAPER+f"tsne_{'temp'}.png")


In [91]:
df

,x,y,color,id,images
0,0.762772,-18.996752,0,0,./local/images/img0.jpeg
1,-13.370202,1.609255,0,1,./local/images/img1.jpeg
2,7.707167,-41.639351,0,2,./local/images/img2.jpeg
3,-17.297344,-7.457629,0,3,./local/images/img3.jpeg
4,-11.736951,43.415127,0,4,./local/images/img4.jpeg
...,...,...,...,...,...
5995,6.965897,-0.107697,0,5995,./local/images/img5995.jpeg
5996,-3.382307,-6.609585,0,5996,./local/images/img5996.jpeg
5997,-43.464920,-8.712676,0,5997,./local/images/img5997.jpeg
5998,22.487162,-4.764453,0,5998,./local/images/img5998.jpeg


In [ ]:
# sudo conda install -c conda-forge dash --name paper2022
# sudo conda install -c conda-forge jupyter-dash --name paper2022

In [92]:
import io
import base64
import pickle
import gzip

import numpy as np

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, no_update
import plotly.graph_objects as go

from PIL import Image

from sklearn.manifold import TSNE

# Helper functions
def np_image_to_base64(im_matrix):
    im = Image.fromarray(im_matrix)
    buffer = io.BytesIO()
    im.save(buffer, format="jpeg")
    encoded_image = base64.b64encode(buffer.getvalue()).decode()
    im_url = "data:image/jpeg;base64, " + encoded_image
    return im_url

In [93]:
color_map = {
    'Positive Samples': "#E52B50",
    'Negative Samples': "#3B7A57",
    0: "#3B7A57",
    3: "#3DDC84",
    4: "#FFBF00",
    5: "#915C83",
    6: "#008000",
    7: "#7FFFD4",
    8: "#E9D66B",
    9: "#007FFF",
}


labels = df['color'].values
colors = [color_map[label] for i,label in enumerate(labels)]

fig = go.Figure(data=[go.Scatter(
    x=X_tsne[:, 0],
    y=X_tsne[:, 1],
    # z=tsne[:, 2],
    mode='markers',
    marker=dict(
        # size=2,
        color=colors,
    )
)])

_=fig.update_layout(
    # autosize=False,
    width=1000,
    height=1000,

)
_=fig.update_traces(
    hoverinfo="none",
    hovertemplate=None,
)
# _=fig.update_layout(
#     scene=dict(
#         xaxis=dict(range=[-10,10]),
#         yaxis=dict(range=[-10,10]),
#         # zaxis=dict(range=[-10,10]),   
#     )
# )

In [94]:
app = JupyterDash(__name__)

app.layout = html.Div(
    className="container",
    children=[
        dcc.Graph(id="graph-5", figure=fig, clear_on_unhover=True),
        dcc.Tooltip(id="graph-tooltip-5", direction='bottom'),
    ],
)

@app.callback(
    Output("graph-tooltip-5", "show"),
    Output("graph-tooltip-5", "bbox"),
    Output("graph-tooltip-5", "children"),
    Input("graph-5", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    hover_data = hoverData["points"][0]
    bbox = hover_data["bbox"]
    num = hover_data["pointNumber"]

    im_matrix = res[num]
    im_url = np_image_to_base64(im_matrix)
    children = [
        html.Div([
            html.Img(
                src=im_url,
                style={"height": "400px", 'display': 'block', 'margin': '0 auto'},
            ),
            # html.P("MNIST Digit " + str(labels[num]), style={'font-weight': 'bold'})
            html.P("MNIST Digit " , style={'font-weight': 'bold'})

        ])
        
    ]

    return True, bbox, children

if __name__ == "__main__":
    app.run_server(mode='inline', debug=True)